# FDIC Data API Client

By 503 Team 4

Source:
https://banks.data.fdic.gov/docs/#/

## Code

Import libraries

In [2]:
import requests
import pandas as pd
import json
import urllib.parse


Define function

In [3]:
# set endpoint
endpoint = 'https://banks.data.fdic.gov/api/'

# function
def get_all_financial_reporting_data(endpoint, reporting_period, fields, result_limit=5000):
    """
    NC (Noninsured non-deposit commercial banks) are removed below to match the query of the FDIC site https://banks.data.fdic.gov/bankfind-suite/financialreporting
    """
    
    
    # initialize data list
    financials = []
    
    # query
    fields_encoded = urllib.parse.quote(fields, safe='')
    url = f'{endpoint}financials?filters=RISDATE:{reporting_period} AND !(BKCLASS:"NC")&fields={fields_encoded}&limit={result_limit}&sort_by=REPDTE&sort_order=ASC'
    
    # get response
    response = requests.get(url)

    if response.status_code == 200:
        # response to json
        data = response.json()

        # store response data
        financial_data = [item['data'] for item in data['data']]
        financials.extend(financial_data)

    else:
        print(f'Request failed with status code {response.status_code}')
        return None

    return pd.DataFrame(financials)


Query the data

In [93]:
# params
reporting_period = '2022-12-31'
fields = 'REPDTE,CERT,NAME,CITY,STNAME,ASSET,DEPDOM,BKCLASS,ADDRESS,ZIP,EQ,LIAB,ESTYMD,ENDEFYMD,' # no spaces between!
result_limit = 50

# call
all_financial_reporting_data = get_all_financial_reporting_data(endpoint, reporting_period, fields, result_limit)

# display
if all_financial_reporting_data is not None:
    all_financial_reporting_data = all_financial_reporting_data.sort_values('CERT')
    display(all_financial_reporting_data.head())
    all_financial_reporting_data.to_csv(f'quarterly_financials/{reporting_period}_quarterly_financials.csv', index=False)


,ZIP,BKCLASS,REPDTE,DEPDOM,ASSET,STNAME,EQ,NAME,CITY,ADDRESS,ENDEFYMD,CERT,ESTYMD,LIAB,ID
7,48858,SM,20221231,1753341,1981367,MICHIGAN,157576,ISABELLA BANK,MOUNT PLEASANT,200 E BROADWAY ST,99991231,1005,19030701,1823791,1005_20221231
12,49230,SM,20221231,101624,113473,MICHIGAN,10543,BANK MICHIGAN,BROOKLYN,417 S MAIN ST,99991231,1008,19071001,102930,1008_20221231
25,49345,SM,20221231,2126308,2382585,MICHIGAN,192540,CHOICEONE BANK,SPARTA,109 E DIVISION ST,99991231,1014,18980101,2190045,1014_20221231
49,53818,N,20221231,306266,331939,WISCONSIN,16877,CLARE BANK NATIONAL ASSN,PLATTEVILLE,345 W PINE ST,99991231,1022,19051028,315062,1022_20221231
0,53946,NM,20221231,123276,148421,WISCONSIN,10659,ERGO BANK,MARKESAN,86 E WATER ST,99991231,10004,19100120,137762,10004_20221231


Check data

In [77]:
display(all_financial_reporting_data.shape)
column_to_check = 'ACTIVE'
display(pd.DataFrame({'col1': all_financial_reporting_data[column_to_check].value_counts().index, 'count': all_financial_reporting_data[column_to_check].value_counts().values}))



(4715, 15)

,col1,count
0,1,4715


Get the last quarter for the past 30 years

In [4]:
today = pd.Timestamp.now().date()


# define the start and end dates of the range (inclusive)
start_date = '1992-03-31'
end_date = '2012-03-31'

# generate a list of dates within the range, with a frequency of 1 quarter
dates = pd.date_range(start=start_date, end=end_date, freq='Q')

# loop
for date in dates:

    # get end of quarter
    end_of_quarter = date.strftime('%Y-%m-%d') # format
    
    fields = 'REPDTE,CERT,NAME,CITY,STNAME,ASSET,DEPDOM,BKCLASS,ADDRESS,ZIP,EQ,LIAB,ESTYMD,ENDEFYMD,' # no spaces between!
    result_limit = 10000

    # call
    all_financial_reporting_data = get_all_financial_reporting_data(endpoint, end_of_quarter, fields, result_limit)

    # display
    if all_financial_reporting_data is not None:
        all_financial_reporting_data = all_financial_reporting_data.sort_values('CERT')
        all_financial_reporting_data.to_csv(f'../../data/quarterly_financials/{end_of_quarter}_quarterly_financials.csv', index=False)
        print(f'{end_of_quarter} done')


1992-03-31 done
